In [2]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("Dataset_comparision.xlsx")

# Display the first few rows to understand the structure
df.head()

,filename,core_id,qmof_id
0,1499489-acs.cgd.6b01265_1499490_clean,1499489-acs.cgd.6b01265_1499490_clean,NaN
1,ABACUF01_FSR,NaN,qmof-8a95c27
2,ABALOF_FSR,NaN,qmof-019ba28
3,ABAVIJ_clean,ABAVIJ_clean,NaN
4,ABAVIJ_FSR,NaN,qmof-830ed1c


In [3]:
# Extract the prefix (string before "_") from the "filename" column
df['prefix'] = df['filename'].apply(lambda x: x.split('_')[0])

# Group by the prefix and filter groups having more than one entry
filtered_df = df[df.groupby('prefix')['filename'].transform('size') > 1]

# Since we're interested in extracting rows with the same prefix, drop the helper column
filtered_df = filtered_df.drop(columns=['prefix'])

filtered_df

,filename,core_id,qmof_id
3,ABAVIJ_clean,ABAVIJ_clean,NaN
4,ABAVIJ_FSR,NaN,qmof-830ed1c
5,ABAVOP_clean,ABAVOP_clean,NaN
6,ABAVOP_FSR,NaN,qmof-5bd4a24
24,ABEXEM_clean,ABEXEM_clean,NaN
...,...,...,...
34499,ZUSBIA_FSR,NaN,qmof-1d483e8
34500,ZUSBOG_clean,ZUSBOG_clean,NaN
34501,ZUSBOG_FSR,NaN,qmof-da4f926
34514,ZUYLOW_clean,ZUYLOW_clean,NaN


In [4]:
# Update the "filename" column to keep only the prefix (string before "_")
filtered_df['filename'] = filtered_df['filename'].apply(lambda x: x.split('_')[0])

# Fill NaN values in 'core_id' and 'qmof_id' columns with the corresponding values where available
# We assume "corresponding values" means filling NaNs in each column with non-NaN values from the same group (if any)
filtered_df['core_id'] = filtered_df.groupby('filename')['core_id'].transform(lambda x: x.ffill().bfill())
filtered_df['qmof_id'] = filtered_df.groupby('filename')['qmof_id'].transform(lambda x: x.ffill().bfill())

filtered_df

,filename,core_id,qmof_id
3,ABAVIJ,ABAVIJ_clean,qmof-830ed1c
4,ABAVIJ,ABAVIJ_clean,qmof-830ed1c
5,ABAVOP,ABAVOP_clean,qmof-5bd4a24
6,ABAVOP,ABAVOP_clean,qmof-5bd4a24
24,ABEXEM,ABEXEM_clean,qmof-780219c
...,...,...,...
34499,ZUSBIA,ZUSBIA_clean,qmof-1d483e8
34500,ZUSBOG,ZUSBOG_clean,qmof-da4f926
34501,ZUSBOG,ZUSBOG_clean,qmof-da4f926
34514,ZUYLOW,ZUYLOW_clean,qmof-2f6a6a9


In [5]:
# Remove duplicate rows
filtered_df_unique = filtered_df.drop_duplicates()

filtered_df_unique

,filename,core_id,qmof_id
3,ABAVIJ,ABAVIJ_clean,qmof-830ed1c
5,ABAVOP,ABAVOP_clean,qmof-5bd4a24
24,ABEXEM,ABEXEM_clean,qmof-780219c
28,ABEXOW,ABEXOW_clean,qmof-4d61b4b
30,ABEXUC,ABEXUC_clean,qmof-d382d2b
...,...,...,...
34464,ZUBKEO,ZUBKEO_clean,qmof-f80342c
34493,ZURLAB,ZURLAB_clean,qmof-40f5c75
34498,ZUSBIA,ZUSBIA_clean,qmof-1d483e8
34500,ZUSBOG,ZUSBOG_clean,qmof-da4f926


In [6]:
# Delete all rows with missing values from filtered_df_unique
filtered_df_no_missing = filtered_df_unique.dropna()

filtered_df_no_missing

,filename,core_id,qmof_id
3,ABAVIJ,ABAVIJ_clean,qmof-830ed1c
5,ABAVOP,ABAVOP_clean,qmof-5bd4a24
24,ABEXEM,ABEXEM_clean,qmof-780219c
28,ABEXOW,ABEXOW_clean,qmof-4d61b4b
30,ABEXUC,ABEXUC_clean,qmof-d382d2b
...,...,...,...
34464,ZUBKEO,ZUBKEO_clean,qmof-f80342c
34493,ZURLAB,ZURLAB_clean,qmof-40f5c75
34498,ZUSBIA,ZUSBIA_clean,qmof-1d483e8
34500,ZUSBOG,ZUSBOG_clean,qmof-da4f926


In [7]:
# Save the filtered and unique DataFrame to a CSV file without the index
filtered_df_no_missing.to_csv("cleaned_dataset.csv", index=False)